In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from IPython.display import display, HTML
%matplotlib inline

def parse_if_number(s):
    try: return float(s)
    except: return True if s=="true" else False if s=="false" else s if s else None

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def get_file_name(name):
    return name.replace(':', '-')

In [2]:
inputFile = 'hightime.csv'
repetitionsCount = -1 # -1 = auto-detect

# Filter scalars
scalarsFilter = ['Floorplan.userCount']
# Filter vectors
vectorsFilter = ['Floorplan.coveredUsers:vector']
# Percentiles
percentiles = [0.9, 0.95, 0.98, 0.99]

intPercentiles = [int(i*100) for i in percentiles]
broadcastTimes = []
for intPercentile in intPercentiles:
    broadcastTimes.append(('broadcastTime' + str(intPercentile), 'Broadcast time needed to reach the ' + str(intPercentile) + 'th percentile of the coverage'))

In [3]:
df = pd.read_csv('exported_data/' + inputFile, converters = {
    'attrvalue': parse_if_number,
    'binedges': parse_ndarray,
    'binvalues': parse_ndarray,
    'vectime': parse_ndarray,
    'vecvalue': parse_ndarray,
})

In [4]:
if repetitionsCount <= 0: # auto-detect
    repetitionsCount = int(df[df.attrname == 'repetition']['attrvalue'].max()) + 1
print('Repetitions:', repetitionsCount)

display(HTML("<style>div.output_scroll { height: auto; max-height: 48em; }</style>"))
pd.set_option('display.max_rows', 1000)

Repetitions: 30


In [5]:
scalars = df[(df.type == 'scalar') | ((df.type == 'itervar') & (df.attrname != 'TO')) | ((df.type == 'param') & (df.attrname == 'Floorplan.userCount')) | ((df.type == 'runattr') & (df.attrname == 'repetition'))]
scalars = scalars.assign(qname = scalars.attrname.combine_first(scalars.module + '.' + scalars.name))
for index, row in scalars[scalars.type == 'itervar'].iterrows():
    val = scalars.loc[index, 'attrvalue']
    if isinstance(val, str) and not all(c.isdigit() for c in val):
        scalars.loc[index, 'attrvalue'] = eval(val)
scalars.value = scalars.value.combine_first(scalars.attrvalue.astype('float64'))
scalars_wide = scalars.pivot_table(index=['run'], columns='qname', values='value')
scalars_wide.sort_values(['X', 'Y', 'repetition'], inplace=True)
for index, row in scalars_wide.iterrows():
    if row['X'] == 0 and row['Y'] == 0:
        pos = 'corner'
    elif row['X'] == 0 or row['Y'] == 0:
        pos = 'border'
    else:
        pos = 'center'
    scalars_wide.loc[index, 'position'] = pos
scalars_wide = scalars_wide[['position', 'repetition', *scalarsFilter]]

In [6]:
vectors = df[df.type == 'vector']
vectors = vectors.assign(qname = vectors.module + '.' + vectors.name)
for index in scalars_wide.index:
    r = index
    pos = scalars_wide.loc[index, 'position']
    rep = scalars_wide.loc[index, 'repetition']
    vectors.loc[vectors.run == r, 'position'] = pos
    vectors.loc[vectors.run == r, 'repetition'] = rep
vectors = vectors[vectors.qname.isin(vectorsFilter)]
vectors.sort_values(['position', 'repetition', 'qname'], inplace=True)
vectors = vectors[['position', 'repetition', 'qname', 'vectime', 'vecvalue']]

In [7]:
def get_percentile(percentile, vectime, vecvalue, totalvalue):
    tofind = percentile * totalvalue
    idx = 0
    csum = vecvalue.cumsum()
    for value in csum:
        if value >= tofind:
            return vectime[idx]
        idx += 1
    return math.inf

for index, row in vectors.iterrows():
    for bt, percentile in zip(broadcastTimes, percentiles):
        btname = bt[0]
        pos = row['position']
        rep = row['repetition']
        total = scalars_wide[(scalars_wide['position'] == pos) & (scalars_wide['repetition'] == rep)]['Floorplan.userCount'].values[0] - 1
        value = get_percentile(percentile, row['vectime'], row['vecvalue'], total)
        scalars_wide.loc[(scalars_wide['position'] == pos) & (scalars_wide['repetition'] == rep), btname] = value
scalars_wide

qname,position,repetition,Floorplan.userCount,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
run,,,,,,,
StartNodePositionHighDensityTime-0-20210614-13:08:30-32563,corner,0.0,1125.0,51.0,54.0,57.0,61.0
StartNodePositionHighDensityTime-1-20210614-13:08:41-32563,corner,1.0,1125.0,50.0,53.0,55.0,59.0
StartNodePositionHighDensityTime-2-20210614-13:08:52-32563,corner,2.0,1125.0,56.0,60.0,62.0,inf
StartNodePositionHighDensityTime-3-20210614-13:09:03-32563,corner,3.0,1125.0,51.0,53.0,60.0,68.0
StartNodePositionHighDensityTime-4-20210614-13:09:14-32563,corner,4.0,1125.0,53.0,56.0,60.0,63.0
StartNodePositionHighDensityTime-5-20210614-13:08:30-32564,corner,5.0,1125.0,50.0,53.0,58.0,62.0
StartNodePositionHighDensityTime-6-20210614-13:08:41-32564,corner,6.0,1125.0,48.0,51.0,56.0,57.0
StartNodePositionHighDensityTime-7-20210614-13:08:52-32564,corner,7.0,1125.0,51.0,55.0,59.0,inf
StartNodePositionHighDensityTime-8-20210614-13:09:03-32564,corner,8.0,1125.0,50.0,53.0,58.0,59.0


In [8]:
for pos in ['center', 'border', 'corner']:
    print('Stats for position', pos)
    display(scalars_wide[scalars_wide['position'] == pos].describe()[[item[0] for item in broadcastTimes]])

Stats for position center


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.000000,30.000000,30.000000,30.00
mean,26.433333,29.000000,32.533333,inf
std,0.626062,0.830455,2.161311,NaN
min,25.000000,28.000000,30.000000,32.00
25%,26.000000,28.000000,31.000000,33.25
50%,26.000000,29.000000,32.000000,35.00
75%,27.000000,29.750000,33.750000,40.25
max,28.000000,31.000000,39.000000,inf


Stats for position border


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.000000,30.000000,30.000000,30.00
mean,41.900000,44.800000,48.666667,inf
std,1.213431,1.242911,2.039833,NaN
min,40.000000,43.000000,45.000000,48.00
25%,41.000000,44.000000,47.000000,50.00
50%,42.000000,45.000000,49.000000,52.00
75%,42.750000,46.000000,50.000000,53.75
max,45.000000,47.000000,53.000000,inf


Stats for position corner


qname,broadcastTime90,broadcastTime95,broadcastTime98,broadcastTime99
count,30.000000,30.000000,30.000000,30.0
mean,50.333333,53.833333,57.700000,inf
std,1.667816,1.876963,1.803254,NaN
min,48.000000,51.000000,55.000000,56.0
25%,49.000000,53.000000,56.000000,59.0
50%,50.000000,53.500000,57.500000,61.5
75%,51.000000,55.000000,59.000000,65.0
max,56.000000,60.000000,62.000000,inf
